# Capstone: Studying the proximity of schools and other venues to Albertan Communities

## Introduction/ Business Problem

40,000 new immigrants make Alberta Canada their home each year. These immigrants can be from other parts of Canada or from other countries.
This study will determine the venues and schools in close proximity to Albertan neighbourhoods.
These neighbourhoods will be grouped by clustering neighbourhoods with similar venues and schools in close proximity.
Immigrants can use this information to find a neighbourhood that meets their venue requirements.

## Description of Data

This study will require the following sources of data:   
 1. Alberta postal codes from ## Description of Data

This study will require the following sources of data:   
 1. Alberta postal codes from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T  
   _This wiki page has all the postal codes for Alberta and includes lat & long coords_  
   
 2. Foursquare API  
   _This API provides the venues and schools in close proximity to the neighbourhoods_   
   _This wiki page has all the postal codes for Alberta and includes lat & long coords_  
   
 2. Foursquare API  
   _This API provides the venues and schools in close proximity to the neighbourhoods_  
 

## This section imports the required libraries

In [1]:
# Import required libraries

import numpy as np # library to handle data in a vectorized manner
import csv

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from bs4 import BeautifulSoup # site scraping library
import requests

# convert an address into latitude and longitude 
from geopy.geocoders import Nominatim
from geopy.distance import geodesic # calculated the distance between two lat,long

# map rendering library
import folium

 # import geocoder
import geocoder

print('Libraries imported.')

Libraries imported.


## Section 1: This section will scrape the wikipedia page for Alberta postal codes

In [2]:
# send HTTP request to a URL
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T'
# make get request
page = requests.get(URL)
# create soup object parsed using beautifulSoup
soup = BeautifulSoup(page.content,'html.parser')



In [3]:
# find wikitable

wiki_table = soup.find("table",attrs={"class": "wikitable"})
wiki_table_header=wiki_table.find("tr")

# Get all the headings of Lists
headings=[]
for th in wiki_table_header.find_all("th"):
   headings.append(th.text.replace('\n',' ').strip())

#optional print headings
#print(headings)

# Get all the rows of table
table_data = []
for tr in wiki_table.tbody.find_all("tr"): # find all tr's from table's tbody
    
    # find all td
    t_row={}
    for td,header in zip(tr.find_all("td"),headings):
        t_row[header]=td.text.replace('\n',' ').strip()
    
    table_data.append(t_row)

#optional print table_data
#print(table_data)

# Put the data for the table with his heading.
alberta_boroughs=pd.DataFrame.from_dict(table_data)
alberta_boroughs.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN
1,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
2,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
3,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
4,T4A,Airdrie,East Airdrie,51.272450,-113.986980


### This section will clean the data to remove NaN values and reset the index

In [4]:
#replace 'not assigned' with NaN
alberta_boroughs.replace('Not assigned',np.NaN,inplace=True)

#drop rows with NaN postal code or borough and reset index
alberta_boroughs.dropna(subset=('Postal Code', 'Borough','Latitude','Longitude'),inplace=True)

#replace blank Neighbourhoods with value from Borough
alberta_boroughs.Neighborhood.fillna(alberta_boroughs.Borough, inplace=True)

#change Lat & Long columns from str to float
alberta_boroughs['Latitude'] = alberta_boroughs['Latitude'].astype(float)
alberta_boroughs['Longitude'] = alberta_boroughs['Longitude'].astype(float)

alberta_boroughs.reset_index(drop=True,inplace=True)
alberta_boroughs.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.03646,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.04968,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12606,-114.143158
3,T4A,Airdrie,East Airdrie,51.27245,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.58990,-113.441300


### Shape of dataframe

In [5]:
alberta_boroughs.shape

(138, 5)

## SECTION 2
# This section will explore and cluster the neighbourhoods in Alberta

In [6]:
#get the lat & long of Alberta Canada
address = 'Calgary, Alberta Canada'

geolocator = Nominatim(user_agent="YYC_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Calgary are {}, {}.'.format(latitude, longitude))


The geograpical coordinates of Calgary are 51.0534234, -114.0625892.


In [7]:
# create map of Alberta with neighbourhoods superimposted using latitude and longitude values
map_ab= folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, borough, neighborhood in zip(alberta_boroughs['Latitude'], alberta_boroughs['Longitude'], alberta_boroughs['Borough'], alberta_boroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ab)  
    
map_ab

#### Define Foursquare Credentials and Verison

In [8]:
CLIENT_ID = 'H33WJJBNAVHQZUX1IQ4GI21QB5P5NIAQI1BX3JXLOAYZPXB0' # your Foursquare ID
CLIENT_SECRET = 'OENW3QCG2ACYUPJPWHU4LLLLZRAMJ5FZYQI4PRXQQHJFSYIJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H33WJJBNAVHQZUX1IQ4GI21QB5P5NIAQI1BX3JXLOAYZPXB0
CLIENT_SECRET:OENW3QCG2ACYUPJPWHU4LLLLZRAMJ5FZYQI4PRXQQHJFSYIJ


In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#read from csv if code not markdown
AB_venues=pd.read_csv('AB_venues_save.csv')

In [10]:
#get venues from foursquare - switch to markdown to disable
AB_venues = getNearbyVenues(names=alberta_boroughs['Neighborhood'],
                                   latitudes=alberta_boroughs['Latitude'],
                                   longitudes=alberta_boroughs['Longitude']
                                  )
AB_venues.to_csv('AB_Venues_save.csv')
print("/n**** SAVED TO .csv ****")

Central Medicine Hat
Penbrooke Meadows, Marlborough
Dalhousie, Edgemont, Hamptons, Hidden Valley
East Airdrie
West Clareview, East Londonderry
North Capilano
Drayton Valley
West Sherwood Park
Wetaskiwin
South Medicine Hat
Forest Lawn, Dover, Erin Woods
Montgomery, Bowness, Silver Springs, Greenwood
West Airdrie
East North Central, West Beverly
SE Capilano, West Southeast Industrial, East Bonnie Doon
Outer Southwest
North Medicine Hat
Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains
Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta
Cochrane
Central Londonderry
Central Bonnie Doon
Inner Southwest Sherwood Park
Vegreville
Bridgeland, Greenview, Zoo, YYC
Lakeview, Glendale, Killarney, Glamorgan
Red Deer County
West Londonderry, East Calder
South Bonnie Doon, East University
Edson
Central Sherwood Park (Ardrossan)
Leduc (Includes YEG)
Taber
Inglewood, Burnsland, Chinatown, East Victoria Park, Saddledome
Hawkwood, Arbour Lake, Citadel, Ranchlands, Royal Oak, Rocky Ridge
Innisfail

In [11]:
print(AB_venues.shape)
AB_venues.head()

(2694, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Central Medicine Hat,50.03646,-110.67925,Local Public Eatery Medicine Hat,50.039218,-110.676133,Pub
1,Central Medicine Hat,50.03646,-110.67925,Madhatter Coffee Roastery,50.039163,-110.677189,Coffee Shop
2,Central Medicine Hat,50.03646,-110.67925,Swirls,50.035248,-110.687189,Ice Cream Shop
3,Central Medicine Hat,50.03646,-110.67925,Esplanade Arts And Heritage Centre,50.039907,-110.680308,Theater
4,Central Medicine Hat,50.03646,-110.67925,Thúy Tiên,50.041009,-110.675177,Vietnamese Restaurant


AB_venues.groupby('Neighborhood').count()

In [12]:
print('There are {} uniques categories.'.format(len(AB_venues['Venue Category'].unique())))

There are 257 uniques categories.


#### Analyze Each Neighborhood

In [13]:
# one hot encoding
AB_onehot = pd.get_dummies(AB_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
AB_onehot['Neighborhood'] = AB_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [AB_onehot.columns[-1]] + list(AB_onehot.columns[:-1])

AB_onehot = AB_onehot[fixed_columns]

AB_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Bed & Breakfast,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Classroom,College Gym,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Lodge,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trade School,Trail,Train Station,Travel & Transport,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Medicine Hat,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [14]:
AB_onehot.shape

(2694, 258)

In [15]:
AB_grouped = AB_onehot.groupby('Neighborhood').mean().reset_index()

In [16]:
AB_grouped.head()

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Bed & Breakfast,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Classroom,College Gym,College Residence Hall,College Theater,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Lodge,Ski Shop,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tea Room,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Trade School,Trail,Train Station,Travel & Transport,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Athabasca,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.2,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.200000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0

num_top_venues = 5

for hood in AB_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = AB_grouped[AB_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = AB_grouped['Neighborhood']

for ind in np.arange(AB_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(AB_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Athabasca,Gas Station,Automotive Shop,Sandwich Place,Burger Joint,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Factory
1,Banff,Hotel,Steakhouse,Restaurant,Italian Restaurant,Pub,Greek Restaurant,Grocery Store,Sporting Goods Shop,Bus Station,Café
2,Barrhead,Grocery Store,Bank,Fast Food Restaurant,Financial or Legal Service,Falafel Restaurant,Farmers Market,Field,Filipino Restaurant,Flea Market,Event Space
3,Beaumont,Coffee Shop,Fast Food Restaurant,Bank,Ice Cream Shop,Pizza Place,Smoothie Shop,French Restaurant,Sandwich Place,Grocery Store,Pharmacy
4,Bonnyville,Coffee Shop,Ice Cream Shop,Sandwich Place,Grocery Store,Gym / Fitness Center,Factory,Convenience Store,Hotel,Fondue Restaurant,Flower Shop


 #### Cluster Neighborhoods using kmeans machine learning algorithm

In [19]:
# set number of clusters
kclusters = 7

AB_grouped_clustering = AB_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(AB_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_)
print(len(kmeans.labels_))


[3 3 0 3 4 3 4 3 4 4 5 3 3 3 3 4 3 3 3 3 3 4 3 3 3 3 4 4 4 3 0 3 3 4 3 3 6
 6 0 3 5 4 4 3 1 3 4 3 5 4 3 3 4 3 3 3 3 4 3 3 4 3 3 3 4 3 0 3 1 3 3 4 3 3
 3 3 3 5 3 3 4 3 3 3 3 4 4 4 4 2 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 0
 3 3 4 4 3 3 1 3 3 3 3 3 3 3 3 3 3 3 1 4 4]
132


In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels',kmeans.labels_)

AB_merged = alberta_boroughs

# merge AB_grouped with alberta_data to add latitude/longitude for each neighborhood
AB_merged = AB_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
AB_merged.dropna(inplace=True)
AB_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,T1A,Medicine Hat,Central Medicine Hat,50.03646,-110.679250,3.0,Coffee Shop,Pub,Park,Bank,Ice Cream Shop,Theater,Italian Restaurant,Electronics Store,Smoke Shop,Supermarket
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.04968,-113.964320,4.0,Convenience Store,Pharmacy,Gym / Fitness Center,Pub,Noodle House,Sandwich Place,Fast Food Restaurant,Golf Course,Bank,Grocery Store
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12606,-114.143158,4.0,Gas Station,Convenience Store,Trail,Asian Restaurant,Yoga Studio,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service
3,T4A,Airdrie,East Airdrie,51.27245,-113.986980,3.0,Fast Food Restaurant,Pub,Pharmacy,Grocery Store,Sushi Restaurant,Gas Station,Gastropub,Chinese Restaurant,Coffee Shop,Bank
4,T5A,Edmonton,"West Clareview, East Londonderry",53.58990,-113.441300,3.0,Buffet,Comedy Club,Mexican Restaurant,Bus Station,Casino,Auto Garage,Toy / Game Store,Convenience Store,Dog Run,Field


#### Add to map

In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(AB_merged['Latitude'], AB_merged['Longitude'], AB_merged['Neighborhood'], AB_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1 - Red


In [22]:
AB_merged.loc[AB_merged['Cluster Labels'] == 0, AB_merged.columns[[1] + [2] + list(range(5, AB_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Edmonton,North Capilano,0.0,Fast Food Restaurant,Restaurant,Park,Fruit & Vegetable Store,Yoga Studio,Falafel Restaurant,Farmers Market,Field,Filipino Restaurant,Financial or Legal Service
6,Drayton Valley,Drayton Valley,0.0,Fast Food Restaurant,Pharmacy,Travel & Transport,Yoga Studio,Financial or Legal Service,Falafel Restaurant,Farmers Market,Field,Filipino Restaurant,Flea Market
29,Edson,Edson,0.0,Fast Food Restaurant,Pizza Place,Coffee Shop,Grocery Store,Breakfast Spot,Inn,Restaurant,Yoga Studio,Filipino Restaurant,Farmers Market
32,Taber,Taber,0.0,Fast Food Restaurant,Convenience Store,Inn,Yoga Studio,Flea Market,Farmers Market,Field,Filipino Restaurant,Financial or Legal Service,Flower Shop
79,Barrhead,Barrhead,0.0,Grocery Store,Bank,Fast Food Restaurant,Financial or Legal Service,Falafel Restaurant,Farmers Market,Field,Filipino Restaurant,Flea Market,Event Space


#### Cluster 2 - Purple
This cluster is neighbourhoods that are close to baseball fields

In [23]:
AB_merged.loc[AB_merged['Cluster Labels'] == 1, AB_merged.columns[[1] + [2] + list(range(5, AB_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Calgary,"Hawkwood, Arbour Lake, Citadel, Ranchlands, Ro...",1.0,Pizza Place,Convenience Store,Gym / Fitness Center,Yoga Studio,Financial or Legal Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant
88,Westlock,Westlock,1.0,Clothing Store,Pizza Place,Yoga Studio,Financial or Legal Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flea Market
100,Edmonton,North Clover Bar,1.0,Pizza Place,Yoga Studio,Factory,Food Truck,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
121,Wainwright,Wainwright,1.0,Convenience Store,Steakhouse,Hockey Arena,Pizza Place,Yoga Studio,Financial or Legal Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field


#### Cluster 3 - Blue

In [24]:
AB_merged.loc[AB_merged['Cluster Labels'] == 2, AB_merged.columns[[1] + [2] + list(range(5, AB_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
135,Redwood Meadows,Redwood Meadows,2.0,Golf Course,Yoga Studio,Football Stadium,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Financial or Legal Service


#### Cluster 4 - Teal

In [25]:
AB_merged.loc[AB_merged['Cluster Labels'] == 3, AB_merged.columns[[1] + [2] + list(range(5, AB_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Medicine Hat,Central Medicine Hat,3.0,Coffee Shop,Pub,Park,Bank,Ice Cream Shop,Theater,Italian Restaurant,Electronics Store,Smoke Shop,Supermarket
3,Airdrie,East Airdrie,3.0,Fast Food Restaurant,Pub,Pharmacy,Grocery Store,Sushi Restaurant,Gas Station,Gastropub,Chinese Restaurant,Coffee Shop,Bank
4,Edmonton,"West Clareview, East Londonderry",3.0,Buffet,Comedy Club,Mexican Restaurant,Bus Station,Casino,Auto Garage,Toy / Game Store,Convenience Store,Dog Run,Field
7,Sherwood Park,West Sherwood Park,3.0,Fast Food Restaurant,Coffee Shop,Pharmacy,American Restaurant,Bank,Lounge,Furniture / Home Store,Bowling Alley,Sandwich Place,Supermarket
9,Medicine Hat,South Medicine Hat,3.0,Fast Food Restaurant,Ice Cream Shop,Pharmacy,Smoothie Shop,Miscellaneous Shop,Sushi Restaurant,Gas Station,Sandwich Place,Restaurant,Food & Drink Shop
11,Calgary,"Montgomery, Bowness, Silver Springs, Greenwood",3.0,Coffee Shop,Clothing Store,Park,Pizza Place,Sporting Goods Shop,Restaurant,Fast Food Restaurant,Pharmacy,Tea Room,Shopping Mall
12,Airdrie,West Airdrie,3.0,Gym,Pharmacy,Coffee Shop,Bank,Ice Cream Shop,Supermarket,Beer Store,Breakfast Spot,Discount Store,Pub
13,Edmonton,"East North Central, West Beverly",3.0,Pizza Place,Fast Food Restaurant,Hockey Arena,Restaurant,Lounge,Music Store,Coffee Shop,Bookstore,Italian Restaurant,Furniture / Home Store
16,Medicine Hat,North Medicine Hat,3.0,Pharmacy,Fast Food Restaurant,Cosmetics Shop,Ice Cream Shop,Pizza Place,Coffee Shop,Sandwich Place,Skating Rink,Grocery Store,Pool
17,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",3.0,Convenience Store,Indian Restaurant,Diner,Restaurant,Clothing Store,Skating Rink,Theater,Inn,Pizza Place,Gym / Fitness Center


#### Cluster 5 - Orange
This last cluster is a neighbourhood close to lawyers

In [26]:
AB_merged.loc[AB_merged['Cluster Labels'] == 4, AB_merged.columns[[1] + [2] + list(range(5, AB_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Calgary,"Penbrooke Meadows, Marlborough",4.0,Convenience Store,Pharmacy,Gym / Fitness Center,Pub,Noodle House,Sandwich Place,Fast Food Restaurant,Golf Course,Bank,Grocery Store
2,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",4.0,Gas Station,Convenience Store,Trail,Asian Restaurant,Yoga Studio,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service
8,Wetaskiwin,Wetaskiwin,4.0,Home Service,Bank,Pharmacy,Yoga Studio,Financial or Legal Service,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant
10,Calgary,"Forest Lawn, Dover, Erin Woods",4.0,Convenience Store,Grocery Store,Smoke Shop,Fried Chicken Joint,Sandwich Place,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Falafel Restaurant,Ethiopian Restaurant
14,Edmonton,"SE Capilano, West Southeast Industrial, East B...",4.0,Convenience Store,Coffee Shop,Yoga Studio,Fast Food Restaurant,Pizza Place,Pharmacy,Gas Station,Liquor Store,Sandwich Place,Greek Restaurant
20,Edmonton,Central Londonderry,4.0,Department Store,Pharmacy,Fast Food Restaurant,Ice Cream Shop,Chocolate Shop,Convenience Store,Pizza Place,Food,Fondue Restaurant,Event Space
31,Leduc,Leduc (Includes YEG),4.0,Home Service,Ice Cream Shop,Liquor Store,Fast Food Restaurant,Grocery Store,Gym / Fitness Center,Performing Arts Venue,Baseball Field,Pharmacy,Convenience Store
35,Innisfail,Innisfail,4.0,Fast Food Restaurant,Grocery Store,Coffee Shop,Sandwich Place,Pharmacy,Convenience Store,Gas Station,Bank,Ice Cream Shop,Motel
39,Devon,Devon,4.0,Grocery Store,Ice Cream Shop,Dog Run,Burger Joint,Dim Sum Restaurant,Falafel Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
40,Lethbridge,North Lethbridge,4.0,Convenience Store,Food & Drink Shop,Hockey Arena,Grocery Store,Ice Cream Shop,Coffee Shop,Yoga Studio,Filipino Restaurant,Farmers Market,Fast Food Restaurant


#### Cluster 6

In [27]:
AB_merged.loc[AB_merged['Cluster Labels'] == 5, AB_merged.columns[[1] + [2] + list(range(5, AB_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Calgary,Northeast Calgary,5.0,Hotel,Sporting Goods Shop,American Restaurant,Coffee Shop,Gym / Fitness Center,Gas Station,Filipino Restaurant,Farmers Market,Fast Food Restaurant,Field
113,Hinton,Hinton,5.0,Hotel,Motel,Restaurant,Yoga Studio,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Flea Market
116,Canmore,Canmore,5.0,Hotel,Gas Station,Resort,Pool,Dog Run,Chocolate Shop,Bed & Breakfast,Restaurant,Brewery,Inn
126,Edmonton,Ellerslie,5.0,Hotel,Food Truck,Rental Car Location,Pizza Place,Steakhouse,Motorcycle Shop,Gas Station,Restaurant,Rest Area,Burger Joint


#### Cluster 7

In [28]:
AB_merged.loc[AB_merged['Cluster Labels'] == 6, AB_merged.columns[[1] + [2] + list(range(5, AB_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,Sherwood Park,East Sherwood Park (Ardrossan),6.0,Construction & Landscaping,Yoga Studio,Factory,Food Truck,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market
87,Edmonton,"East Southeast Industrial, South Clover Bar",6.0,Construction & Landscaping,Housing Development,Mobile Phone Shop,Yoga Studio,Flea Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service


#### Cluster 8

In [29]:
AB_merged.loc[AB_merged['Cluster Labels'] == 7, AB_merged.columns[[1] + [2] + list(range(5, AB_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


## SECTION 3
### Collecting Specific School Info

In [30]:
def getNearbySchools(names, latitudes, longitudes, radius=10000):
    LIMIT=100
    categoryId='4f4533804b9074f6e4fb0105,4f4533814b9074f6e4fb0106,4bf58dd8d48988d13d941735' # categories for elementary school, middle school and high school
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
            

        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'School', 
                  'School Latitude', 
                  'School Longitude', 
                  'School Category']
    
            
    return(nearby_venues)

In [31]:
def distance(row):
    add1 = (row['Neighborhood Latitude'], row['Neighborhood Longitude']) 
    add2 = (row['School Latitude'], row['School Longitude']) 
    try: 
        return( geodesic(add1, add2).kilometers )
    except ValueError: return np.nan

#read from csv if not in markdown
AB_Schools=pd.read_csv('AB_Schools_save.csv')

In [32]:
#get nearby schools from foursquare when not in markdown
AB_Schools = getNearbySchools(names=alberta_boroughs['Neighborhood'],
                                   latitudes=alberta_boroughs['Latitude'],
                                   longitudes=alberta_boroughs['Longitude']
                                  )
AB_Schools['Distance km'] = AB_Schools.apply(lambda row: distance(row), axis = 1 )
AB_Schools.to_csv('AB_Schools_save.csv')
print("/n**** SAVED TO .csv ****")

Central Medicine Hat
Penbrooke Meadows, Marlborough
Dalhousie, Edgemont, Hamptons, Hidden Valley
East Airdrie
West Clareview, East Londonderry
North Capilano
Drayton Valley
West Sherwood Park
Wetaskiwin
South Medicine Hat
Forest Lawn, Dover, Erin Woods
Montgomery, Bowness, Silver Springs, Greenwood
West Airdrie
East North Central, West Beverly
SE Capilano, West Southeast Industrial, East Bonnie Doon
Outer Southwest
North Medicine Hat
Lynnwood Ridge, Ogden, Foothills Industrial, Great Plains
Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta
Cochrane
Central Londonderry
Central Bonnie Doon
Inner Southwest Sherwood Park
Vegreville
Bridgeland, Greenview, Zoo, YYC
Lakeview, Glendale, Killarney, Glamorgan
Red Deer County
West Londonderry, East Calder
South Bonnie Doon, East University
Edson
Central Sherwood Park (Ardrossan)
Leduc (Includes YEG)
Taber
Inglewood, Burnsland, Chinatown, East Victoria Park, Saddledome
Hawkwood, Arbour Lake, Citadel, Ranchlands, Royal Oak, Rocky Ridge
Innisfail

In [44]:
print(AB_Schools.shape)
AB_Schools.head()

(2281, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,School,School Latitude,School Longitude,School Category,Distance km
0,Central Medicine Hat,50.03646,-110.67925,Alexandra Jr High,50.033020,-110.674329,High School,0.520324
1,Central Medicine Hat,50.03646,-110.67925,Medicine Hat High School,50.033314,-110.690308,High School,0.866096
2,Central Medicine Hat,50.03646,-110.67925,St. Francis Xavier School,50.050052,-110.678296,Elementary School,1.513376
3,Central Medicine Hat,50.03646,-110.67925,Monsignor McCoy High School,50.050162,-110.681461,High School,1.532279
4,Central Medicine Hat,50.03646,-110.67925,Vincent Massey School,50.051499,-110.694033,Elementary School,1.979788


In [45]:
# one hot encoding
AB_Schools_onehot = pd.get_dummies(AB_Schools[['School Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
AB_Schools_onehot['Neighborhood'] = AB_Schools['Neighborhood'] 
AB_Schools_onehot['Distance km'] = AB_Schools['Distance km'] 

AB_Schools_onehot.drop(columns=['College Academic Building','School','Student Center'],inplace=True)


AB_Schools_onehot =AB_Schools_onehot[['Neighborhood','Elementary School','Middle School','High School','Distance km']]

AB_Schools_onehot.head()

,Neighborhood,Elementary School,Middle School,High School,Distance km
0,Central Medicine Hat,0,0,1,0.520324
1,Central Medicine Hat,0,0,1,0.866096
2,Central Medicine Hat,1,0,0,1.513376
3,Central Medicine Hat,0,0,1,1.532279
4,Central Medicine Hat,1,0,0,1.979788


In [46]:
AB_Schools_Distance=AB_Schools_onehot[['Elementary School','Middle School', 'High School']].multiply(AB_Schools_onehot['Distance km'],axis='index')
AB_Schools_Distance['Neighborhood'] = AB_Schools_onehot['Neighborhood'] 
AB_Schools_Distance =AB_Schools_Distance[['Neighborhood','Elementary School','Middle School','High School']]
AB_Schools_Distance_grouped = AB_Schools_Distance.groupby('Neighborhood').mean().reset_index()
AB_Schools_Distance_grouped.rename(columns={"Elementary School": "E_School_Distance", "Middle School": "M_School_Distance","High School": "H_School_Distance"},inplace='True')
AB_Schools_Distance_grouped.head()

,Neighborhood,E_School_Distance,M_School_Distance,H_School_Distance
0,Athabasca,0.000000,0.000000,0.797252
1,Banff,0.152704,0.000000,0.039474
2,Beaumont,4.229143,2.493393,0.284191
3,Bonnyville,0.063136,0.612501,0.131569
4,"Braeside, Cedarbrae, Woodbine",2.283781,1.105166,1.369389


#### Combine venue and School info

#set as markdown to to not combine
AB_grouped = pd.merge(left=AB_grouped, right=AB_Schools_Distance_grouped, how='left', left_on='Neighborhood', right_on='Neighborhood')
AB_grouped.dropna(inplace=True) #remove rows with na
AB_grouped.reset_index(inplace=True,drop=True)

#preprocess and scale distances for kmeans
scaler = StandardScaler()
AB_grouped[['E_School_Distance','M_School_Distance', 'H_School_Distance']] = scaler.fit_transform(AB_grouped[['E_School_Distance','M_School_Distance', 'H_School_Distance']])

AB_grouped.head()

In [47]:
# set number of clusters
kclusters = 4

AB_grouped_schools_clustering = AB_Schools_Distance_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(AB_grouped_schools_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_)
print(len(kmeans.labels_))


[1 1 2 1 3 3 3 1 1 0 0 1 0 0 1 0 1 2 2 3 1 1 2 3 2 3 1 3 2 0 1 0 1 0 0 2 0
 1 3 0 3 2 0 0 0 1 3 1 0 3 0 1 1 0 3 3 1 1 2 3 2 2 3 1 3 0 0 0 0 0 1 2 3 0
 0 0 3 3 0 0 3 1 1 0 0 1 2 1 2 3 1 3 3 0 3 0 3 0 3 0 3 1 0 0 0 3 0 0 1 3 1
 0 0 3 0 1 1 1 0 0 0 0 0 0 0 0 1 3 1 1 1]
131


In [48]:
# add clustering labels
AB_Schools_Distance_grouped.insert(0, 'Cluster Labels',kmeans.labels_)

AB_schools_merged = AB_Schools_Distance_grouped

# merge AB_grouped with alberta_data to add latitude/longitude for each neighborhood
AB_schools_merged = AB_schools_merged.join(alberta_boroughs.set_index('Neighborhood'), on='Neighborhood')
AB_schools_merged.dropna(inplace=True)
AB_schools_merged=AB_schools_merged[['Cluster Labels','Borough','Neighborhood','Latitude','Longitude','E_School_Distance','M_School_Distance','H_School_Distance']]
AB_schools_merged.head()

,Cluster Labels,Borough,Neighborhood,Latitude,Longitude,E_School_Distance,M_School_Distance,H_School_Distance
0,1,Athabasca,Athabasca,54.7139,-113.2942,0.000000,0.000000,0.797252
1,1,Banff,Banff,51.1791,-115.5697,0.152704,0.000000,0.039474
2,2,Beaumont,Beaumont,53.3571,-113.4129,4.229143,2.493393,0.284191
3,1,Bonnyville,Bonnyville,54.2678,-110.7324,0.063136,0.612501,0.131569
4,3,Calgary,"Braeside, Cedarbrae, Woodbine",50.9604,-114.1001,2.283781,1.105166,1.369389


In [49]:
# create map
map_clusters_2 = folium.Map(location=[latitude, longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(AB_schools_merged['Latitude'], AB_schools_merged['Longitude'], AB_schools_merged['Neighborhood'], AB_schools_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters_2)
       
map_clusters_2

In [50]:
AB_schools_merged.loc[AB_schools_merged['Cluster Labels'] == 0, AB_schools_merged.columns[list(range(1, AB_schools_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,E_School_Distance,M_School_Distance,H_School_Distance
9,Edmonton,Central Beverly,53.5766,-113.4608,0.608558,0.165670,3.754366
10,Edmonton,Central Bonnie Doon,53.5182,-113.4769,0.502166,1.282933,4.038784
12,Edmonton,"Central Jasper Place, Buena Vista",53.5224,-113.5763,0.481593,0.803809,4.564409
13,Edmonton,Central Londonderry,53.6129,-113.4572,0.659195,0.252052,4.733531
15,Edmonton,Central Mistatim,53.5800,-113.5873,0.587053,0.500463,6.024856
29,Drayton Valley,Drayton Valley,53.2165,-114.9893,0.000000,0.000000,4.915790
31,Edmonton,East Castledowns,53.6072,-113.5183,0.603790,0.124012,5.059931
33,Edmonton,East Mill Woods,53.4681,-113.4339,0.387177,0.689415,4.490273
34,Edmonton,"East North Central, West Beverly",53.5766,-113.4608,0.608558,0.165670,3.754366
36,Edmonton,"East Southeast Industrial, South Clover Bar",53.4996,-113.3678,0.758365,0.719354,5.123974


In [51]:
AB_schools_merged.loc[AB_schools_merged['Cluster Labels'] == 1, AB_schools_merged.columns[list(range(1, AB_schools_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,E_School_Distance,M_School_Distance,H_School_Distance
0,Athabasca,Athabasca,54.71390,-113.29420,0.000000,0.000000,0.797252
1,Banff,Banff,51.17910,-115.56970,0.152704,0.000000,0.039474
3,Bonnyville,Bonnyville,54.26780,-110.73240,0.063136,0.612501,0.131569
7,Camrose,Camrose,53.02040,-112.81290,1.229006,0.000000,0.564527
8,Canmore,Canmore,51.08680,-115.33840,0.000000,0.735388,1.105941
11,Grande Prairie,Central Grande Prairie,55.17260,-118.79970,1.024552,0.000000,0.951065
14,Medicine Hat,Central Medicine Hat,50.03646,-110.67925,1.941550,0.967861,0.324300
16,Red Deer,Central Red Deer,52.25920,-113.82370,1.479256,0.000000,1.132113
20,Coaldale,Coaldale,49.72850,-112.61460,0.000000,0.000000,0.870365
21,Cochrane,Cochrane,51.18960,-114.47740,0.000000,0.000000,2.036622


In [52]:
AB_schools_merged.loc[AB_schools_merged['Cluster Labels'] == 2, AB_schools_merged.columns[list(range(1, AB_schools_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,E_School_Distance,M_School_Distance,H_School_Distance
2,Beaumont,Beaumont,53.35710,-113.41290,4.229143,2.493393,0.284191
17,Sherwood Park,Central Sherwood Park (Ardrossan),53.45480,-113.04980,6.594026,0.000000,0.000000
18,Chestermere,Chestermere,51.05120,-113.81550,6.022378,2.969746,0.000000
22,Cold Lake,Cold Lake,54.41270,-110.21620,4.038147,0.000000,0.117064
24,Calgary,"Cranston, Auburn Bay, Mahogany",50.89020,-113.98920,3.532278,1.625616,0.828633
28,Calgary,"Douglas Glen, McKenzie Lake, Copperfield, East...",50.90230,-113.98730,3.265321,1.297884,1.093707
35,Sherwood Park,East Sherwood Park (Ardrossan),53.47490,-112.95120,6.398795,0.000000,0.000000
41,Fort Saskatchewan,Fort Saskatchewan,53.69160,-113.22860,2.674408,0.438800,0.471659
58,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",50.98780,-114.00010,3.596114,1.561871,1.879989
60,Calgary,"Midnapore, Sundance",50.92040,-114.06740,3.000303,0.985107,0.931740


In [53]:
AB_schools_merged.loc[AB_schools_merged['Cluster Labels'] == 3, AB_schools_merged.columns[list(range(1, AB_schools_merged.shape[1]))]]

,Borough,Neighborhood,Latitude,Longitude,E_School_Distance,M_School_Distance,H_School_Distance
4,Calgary,"Braeside, Cedarbrae, Woodbine",50.96040,-114.100100,2.283781,1.105166,1.369389
5,Calgary,"Brentwood, Collingwood, Nose Hill",51.09170,-114.112700,2.254655,0.509795,3.455776
6,Calgary,"Bridgeland, Greenview, Zoo, YYC",51.06320,-114.061400,2.408301,1.193246,2.977905
19,Calgary,"City Centre, Calgary Tower",51.04720,-114.080200,1.918674,1.403903,3.187834
23,Calgary,"Connaught, West Victoria Park",51.04260,-114.079100,2.290488,1.371960,2.754934
25,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.12606,-114.143158,2.076375,0.466363,3.014986
27,Calgary,"Discovery Ridge, Signal Hill, West Springs, Ch...",51.05660,-114.181500,2.634248,0.305684,3.241905
38,Calgary,"Elbow Park, Britannia, Parkhill, Mission",51.01710,-114.081200,2.625320,1.072010,2.429720
40,Calgary,"Forest Lawn, Dover, Erin Woods",51.03180,-113.978600,2.803803,1.172475,2.216096
46,Calgary,"Highfield, Burns Industrial",50.98570,-114.063100,2.737047,1.218922,1.732404
